导入相关的包
pip install pandas
pip install ddddocr
pip install selenium

In [71]:
import time
import ddddocr
from selenium.common import NoSuchElementException, InvalidSelectorException
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd


In [72]:
"""加载OCR，识别验证码用"""
ocr = ddddocr.DdddOcr(show_ad=False)

In [86]:
"""加载客户清单"""
customers = pd.read_excel("tmp/customers.xlsx")
searchResult = pd.DataFrame(columns=['身份证', '法院名称', '地区名称', '立案时间', '案号',
                                     '做出执行依据单位', '生效法律文书确定的义务',
                                     '被执行人的履行情况', '失信被执行人行为具体情形'])

with open("tmp/pos", "r") as f:
    pos = str(f.read())
if len(pos) == 0:
    pos = 2
else:
    pos = int(pos)
print("从第 %d 行开始查询：" % pos)
pos -= 2


从第 2 行开始查询：


In [74]:
"""selenium 启用缓存"""
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'

In [87]:
disruptCount = 0

driver = webdriver.Chrome(chrome_options=chrome_options)
for i in range(pos, customers.shape[0]):
    idcard = customers[customers.columns[0]][i]
    # Navigate to url
    driver.get("http://zxgk.court.gov.cn/shixin/")

    # Enter "webdriver" text and perform "ENTER" keyboard action
    pageOK = False
    while not pageOK:
        try:
            driver.find_element(By.NAME, "pCardNum").send_keys(idcard)
            pageOK = True
        except NoSuchElementException:
            print("cannot open website, sleep 1 minutes")
            time.sleep(60)

    while True:
        driver.find_element_by_id("captchaImg").click()
        time.sleep(2)
        img = driver.find_element_by_id("captchaImg")

        with open('tmp/1.png', 'wb') as file:
            file.write(img.screenshot_as_png)

        with open('tmp/1.png', 'rb') as f:
            img_bytes = f.read()
        check_code = ocr.classification(img_bytes)
        driver.find_element_by_id("yzm").send_keys(check_code)
        time.sleep(1)
        try:
            yzmResult = driver.find_element(By.XPATH, "//*[contains(text(), '验证码正确')]")
            break
        except NoSuchElementException:
            print("验证码错误，重新验证")
            continue

    driver.find_element(By.XPATH, '//button[normalize-space()="查询"]').click()
    time.sleep(5)
    try:
        resultList = driver.find_elements(By.XPATH, '//a[normalize-space()]="查看"')
        if len(resultList) > 0:
            for result in resultList:
                result.click()
                courtNameDetail = driver.find_element_by_id("courtNameDetail").text
                areaNameDetail = driver.find_element_by_id("areaNameDetail").text
                regDateDetail = driver.find_element_by_id("regDateDetail").text
                caseCodeDetail = driver.find_element_by_id("caseCodeDetail").text
                gistUnitDetail = driver.find_element_by_id("gistUnitDetail").text
                dutyDetail = driver.find_element_by_id("dutyDetail").text
                performanceDetail = driver.find_element_by_id("performanceDetail").text
                disruptTypeNameDetail = driver.find_element_by_id("disruptTypeNameDetail").text
                driver.find_element(By.XPATH, '//button[normalize-space()="关闭"]').click()

                print(idcard, courtNameDetail, areaNameDetail, areaNameDetail, regDateDetail, caseCodeDetail,
                      gistUnitDetail, performanceDetail, disruptTypeNameDetail)

                searchResult.loc[disruptCount] = [idcard, courtNameDetail, areaNameDetail, regDateDetail,
                                                  caseCodeDetail, gistUnitDetail, dutyDetail, performanceDetail, disruptTypeNameDetail]
                disruptCount += 1
    except InvalidSelectorException:
        print("%s is ok" % idcard)
    except NoSuchElementException:
        print("%s is ok" % idcard)

    with open("tmp/pos", "w") as f:
        f.write(str(i+2))

    time.sleep(60)

with open("tmp/pos", "w") as f:
    f.write("2")
searchResult.to_excel("tmp/查询结果.xlsx", index=None)
driver.close()

331022198206210102 is ok
332626196108131160 is ok
91331022355336486K is ok
